In [1]:
pip install flask pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [20]:

from flask import Flask, render_template, request, send_file
import pandas as pd
import os

app = Flask(__name__)

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str)
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)

# 필수 열 정의 (열 이름을 실제 데이터프레임의 열 이름으로 수정)
required_columns = ['고객번호', '청구마스터번호', '계약번호']

# 데이터프레임 병합 함수 정의
def merge_dataframes(master_df, *sub_dfs):
    merged_df = master_df.copy()
    for sub_df in sub_dfs:
        existing_columns = [col for col in required_columns if col in master_df.columns and col in sub_df.columns]
        if existing_columns:
            merged_df = pd.merge(merged_df, sub_df, on=existing_columns, how='left')
        else:
            print(f"Warning: No matching columns found for merging with {sub_df}")
    return merged_df

# 데이터프레임 병합
df_combined = merge_dataframes(df_master, df_sub1)
# df_combined = merge_dataframes(df_master, df_sub1, df_sub2, df_sub3, df_sub4)

@app.route('/')
def index():
    columns = df_combined.columns.tolist()
    return render_template('index.html', columns=columns)

@app.route('/filter', methods=['POST'])
def filter_data():
    customer_number = request.form.get('customer_number')
    master_number = request.form.get('master_number')
    contract_number = request.form.get('contract_number')
    selected_columns = request.form.getlist('columns')

    condition = True
    if customer_number:
        condition &= df_combined['고객번호'].str.contains(customer_number, case=False)
    if master_number:
        condition &= df_combined['청구마스터번호'].str.contains(master_number, case=False)
    if contract_number:
        condition &= df_combined['계약번호'].str.contains(contract_number, case=False)

    filtered_data = df_combined.loc[condition, selected_columns]

    return filtered_data.to_html(classes='styledTable', index=False)

@app.route('/download', methods=['POST'])
def download():
    selected_columns = request.form.getlist('columns')
    condition = True
    if request.form.get('customer_number'):
        condition &= df_combined['고객번호'].str.contains(request.form.get('customer_number'), case=False)
    if request.form.get('master_number'):
        condition &= df_combined['청구마스터번호'].str.contains(request.form.get('master_number'), case=False)
    if request.form.get('contract_number'):
        condition &= df_combined['계약번호'].str.contains(request.form.get('contract_number'), case=False)

    filtered_data = df_combined.loc[condition, selected_columns]
    output_file = 'filtered_data.xlsx'
    filtered_data.to_excel(output_file, index=False)

    return send_file(output_file, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)


Master Columns: Index(['순번', '미납관리본부', '미납관리부서', '현행관리본부', '현행관리지사', '청구마스터번호', '고객번호', '계약번호',
       '청구멤버고객명', '청구멤버상호명', '서비스개시일', '신규/유지', '고객상태', '수납방법', '선후납', '설치주소',
       '영업관리번호', '영업자', '체납개월', 'BM', ' 체납액 ', ' 회수액 ', ' 잔존체납액 '],
      dtype='object')
Sub1 Columns: Index(['0', '1', '2', '3', '4', '5', '6', '7', '8'], dtype='object')
0          실입금일자      청구번호          상호     입금액          은행명            계좌번호   
1     2024.01.01  41214046   탁종합관리주식회사  165000         국민은행  60959014637214   
2     2024.01.01  41232935  (주)동인아이티에스   83600         국민은행  60959014718719   
3     2024.01.01  41403840      베로니스명가   44000         국민은행  61039014519158   
4     2024.01.01  41638416  세븐일레븐일산태영점   58015         국민은행  48359073374152   
...          ...       ...         ...     ...          ...             ...   
8210  2024.05.26  41566740    노틀담 몬테소리   83600         하나은행  35592382076937   
8211  2024.05.26  41603925          더빌   66000         국민은행  48359073247902   
8212  2024.05.26  41

Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
On macOS, try disabling the 'AirPlay Receiver' service from System Preferences -> Sharing.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [9]:

import pandas as pd

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str)
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)
# Uncomment these lines to check columns for additional dataframes
# print("Sub2 Columns:", df_sub2.columns)
# print("Sub3 Columns:", df_sub3.columns)
# print("Sub4 Columns:", df_sub4.columns)


Master Columns: Index(['순번', '미납관리본부', '미납관리부서', '현행관리본부', '현행관리지사', '청구마스터번호', '고객번호', '계약번호',
       '청구멤버고객명', '청구멤버상호명', '서비스개시일', '신규/유지', '고객상태', '수납방법', '선후납', '설치주소',
       '영업관리번호', '영업자', '체납개월', 'BM', ' 체납액 ', ' 회수액 ', ' 잔존체납액 '],
      dtype='object')
Sub1 Columns: Index(['0', '1', '2', '3', '4', '5', '6', '7', '8'], dtype='object')


In [12]:
아래는 Python 코드만 포함된 형태로, Flask 애플리케이션과 데이터프레임 열 이름을 확인하는 코드입니다. 코드를 텍스트 설명과 분리하여 제공합니다.

### 열 이름 확인 코드

먼저 열 이름을 확인하는 코드를 실행하여 각 데이터프레임의 열 이름을 확인합니다:

```python
import pandas as pd

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str, header=1)  # 2번째 행을 헤더로 사용
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)
# Uncomment these lines to check columns for additional dataframes
# print("Sub2 Columns:", df_sub2.columns)
# print("Sub3 Columns:", df_sub3.columns)
# print("Sub4 Columns:", df_sub4.columns)
```

이 코드를 실행하여 각 데이터프레임의 열 이름을 확인한 후, 병합할 열 이름을 결정하십시오.

### 웹 애플리케이션 코드

아래는 웹 애플리케이션을 위한 전체 코드입니다. 이 코드는 데이터프레임을 병합하고, 필터링하고, 결과를 HTML로 표시하며, 엑셀 파일로 다운로드할 수 있는 Flask 애플리케이션을 제공합니다.

```python
from flask import Flask, render_template, request, send_file
import pandas as pd
import os

app = Flask(__name__)

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str, header=1)  # 2번째 행을 헤더로 사용
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)

# 필수 열 정의 (열 이름을 실제 데이터프레임의 열 이름으로 수정)
required_columns = ['고객번호', '청구마스터번호', '계약번호']

# 데이터프레임 병합 함수 정의
def merge_dataframes(master_df, *sub_dfs):
    merged_df = master_df.copy()
    for sub_df in sub_dfs:
        existing_columns = [col for col in required_columns if col in master_df.columns and col in sub_df.columns]
        if existing_columns:
            merged_df = pd.merge(merged_df, sub_df, on=existing_columns, how='left')
        else:
            print(f"Warning: No matching columns found for merging with {sub_df}")
    return merged_df

# 데이터프레임 병합
df_combined = merge_dataframes(df_master, df_sub1)
# df_combined = merge_dataframes(df_master, df_sub1, df_sub2, df_sub3, df_sub4)

@app.route('/')
def index():
    columns = df_combined.columns.tolist()
    return render_template('index.html', columns=columns)

@app.route('/filter', methods=['POST'])
def filter_data():
    customer_number = request.form.get('customer_number')
    master_number = request.form.get('master_number')
    contract_number = request.form.get('contract_number')
    selected_columns = request.form.getlist('columns')

    condition = True
    if customer_number:
        condition &= df_combined['고객번호'].str.contains(customer_number, case=False)
    if master_number:
        condition &= df_combined['청구마스터번호'].str.contains(master_number, case=False)
    if contract_number:
        condition &= df_combined['계약번호'].str.contains(contract_number, case=False)

    filtered_data = df_combined.loc[condition, selected_columns]

    return filtered_data.to_html(classes='styledTable', index=False)

@app.route('/download', methods=['POST'])
def download():
    selected_columns = request.form.getlist('columns')
    condition = True
    if request.form.get('customer_number'):
        condition &= df_combined['고객번호'].str.contains(request.form.get('customer_number'), case=False)
    if request.form.get('master_number'):
        condition &= df_combined['청구마스터번호'].str.contains(request.form.get('master_number'), case=False)
    if request.form.get('contract_number'):
        condition &= df_combined['계약번호'].str.contains(request.form.get('contract_number'), case=False)

    filtered_data = df_combined.loc[condition, selected_columns]
    output_file = 'filtered_data.xlsx'
    filtered_data.to_excel(output_file, index=False)

    return send_file(output_file, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)
```

위 코드를 사용하여 먼저 데이터프레임의 열 이름을 확인하고, 필요한 열 이름을 기반으로 병합 작업을 수행할 수 있습니다. 그런 다음, Flask 애플리케이션을 실행하여 데이터를 필터링하고 다운로드할 수 있습니다.

SyntaxError: invalid syntax (3576989338.py, line 1)

In [13]:
이전 메시지에서 텍스트 설명과 코드가 함께 제공되어 SyntaxError가 발생한 것 같습니다. 이번에는 텍스트 설명을 포함하지 않고, 순수한 Python 코드만 제공하겠습니다. 

먼저 데이터프레임의 열 이름을 확인하는 코드와 Flask 애플리케이션 코드를 분리해서 제공하겠습니다.

### 열 이름 확인 코드

```python
import pandas as pd

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str, header=1)  # 2번째 행을 헤더로 사용
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)
# Uncomment these lines to check columns for additional dataframes
# print("Sub2 Columns:", df_sub2.columns)
# print("Sub3 Columns:", df_sub3.columns)
# print("Sub4 Columns:", df_sub4.columns)
```

### 웹 애플리케이션 코드

```python
from flask import Flask, render_template, request, send_file
import pandas as pd
import os

app = Flask(__name__)

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str, header=1)  # 2번째 행을 헤더로 사용
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)

# 필수 열 정의 (열 이름을 실제 데이터프레임의 열 이름으로 수정)
required_columns = ['고객번호', '청구마스터번호', '계약번호']

# 데이터프레임 병합 함수 정의
def merge_dataframes(master_df, *sub_dfs):
    merged_df = master_df.copy()
    for sub_df in sub_dfs:
        existing_columns = [col for col in required_columns if col in master_df.columns and col in sub_df.columns]
        if existing_columns:
            merged_df = pd.merge(merged_df, sub_df, on=existing_columns, how='left')
        else:
            print(f"Warning: No matching columns found for merging with {sub_df}")
    return merged_df

# 데이터프레임 병합
df_combined = merge_dataframes(df_master, df_sub1)
# df_combined = merge_dataframes(df_master, df_sub1, df_sub2, df_sub3, df_sub4)

@app.route('/')
def index():
    columns = df_combined.columns.tolist()
    return render_template('index.html', columns=columns)

@app.route('/filter', methods=['POST'])
def filter_data():
    customer_number = request.form.get('customer_number')
    master_number = request.form.get('master_number')
    contract_number = request.form.get('contract_number')
    selected_columns = request.form.getlist('columns')

    condition = True
    if customer_number:
        condition &= df_combined['고객번호'].str.contains(customer_number, case=False)
    if master_number:
        condition &= df_combined['청구마스터번호'].str.contains(master_number, case=False)
    if contract_number:
        condition &= df_combined['계약번호'].str.contains(contract_number, case=False)

    filtered_data = df_combined.loc[condition, selected_columns]

    return filtered_data.to_html(classes='styledTable', index=False)

@app.route('/download', methods=['POST'])
def download():
    selected_columns = request.form.getlist('columns')
    condition = True
    if request.form.get('customer_number'):
        condition &= df_combined['고객번호'].str.contains(request.form.get('customer_number'), case=False)
    if request.form.get('master_number'):
        condition &= df_combined['청구마스터번호'].str.contains(request.form.get('master_number'), case=False)
    if request.form.get('contract_number'):
        condition &= df_combined['계약번호'].str.contains(request.form.get('contract_number'), case=False)

    filtered_data = df_combined.loc[condition, selected_columns]
    output_file = 'filtered_data.xlsx'
    filtered_data.to_excel(output_file, index=False)

    return send_file(output_file, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)
```

이제 Python 코드만 포함된 형태로 제공합니다. 열 이름을 먼저 확인한 후 Flask 애플리케이션을 실행하여 데이터를 필터링하고 다운로드할 수 있습니다.

SyntaxError: invalid syntax (3868957828.py, line 1)

In [14]:
import pandas as pd

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str, header=1)  # 2번째 행을 헤더로 사용
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)
# Uncomment these lines to check columns for additional dataframes
# print("Sub2 Columns:", df_sub2.columns)
# print("Sub3 Columns:", df_sub3.columns)
# print("Sub4 Columns:", df_sub4.columns)


Master Columns: Index(['순번', '미납관리본부', '미납관리부서', '현행관리본부', '현행관리지사', '청구마스터번호', '고객번호', '계약번호',
       '청구멤버고객명', '청구멤버상호명', '서비스개시일', '신규/유지', '고객상태', '수납방법', '선후납', '설치주소',
       '영업관리번호', '영업자', '체납개월', 'BM', ' 체납액 ', ' 회수액 ', ' 잔존체납액 '],
      dtype='object')
Sub1 Columns: Index(['실입금일자', '청구번호', '상호', '입금액', '은행명', '계좌번호', '입금자명', '청구본부', '청구지사'], dtype='object')


In [16]:
export FLASK_APP=app.py
export FLASK_ENV=development
flask run


SyntaxError: invalid syntax (2640390298.py, line 1)

In [17]:
from flask import Flask, render_template, request, send_file
import pandas as pd
import os

app = Flask(__name__)

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str, header=1)  # 2번째 행을 헤더로 사용
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)

# 필수 열 정의 (열 이름을 실제 데이터프레임의 열 이름으로 수정)
required_columns = ['고객번호', '청구번호', '계약번호']

# 데이터프레임 병합 함수 정의
def merge_dataframes(master_df, *sub_dfs):
    merged_df = master_df.copy()
    for sub_df in sub_dfs:
        existing_columns = [col for col in required_columns if col in master_df.columns and col in sub_df.columns]
        if existing_columns:
            merged_df = pd.merge(merged_df, sub_df, on=existing_columns, how='left')
        else:
            print(f"Warning: No matching columns found for merging with {sub_df}")
    return merged_df

# 데이터프레임 병합
df_combined = merge_dataframes(df_master, df_sub1)
# df_combined = merge_dataframes(df_master, df_sub1, df_sub2, df_sub3, df_sub4)

@app.route('/')
def index():
    columns = df_combined.columns.tolist()
    return render_template('index.html', columns=columns)

@app.route('/filter', methods=['POST'])
def filter_data():
    customer_number = request.form.get('customer_number')
    master_number = request.form.get('master_number')
    contract_number = request.form.get('contract_number')
    selected_columns = request.form.getlist('columns')

    condition = True
    if customer_number:
        condition &= df_combined['고객번호'].str.contains(customer_number, case=False)
    if master_number:
        condition &= df_combined['청구번호'].str.contains(master_number, case=False)
    if contract_number:
        condition &= df_combined['계약번호'].str.contains(contract_number, case=False)

    filtered_data = df_combined.loc[condition, selected_columns]

    return filtered_data.to_html(classes='styledTable', index=False)

@app.route('/download', methods=['POST'])
def download():
    selected_columns = request.form.getlist('columns')
    condition = True
    if request.form.get('customer_number'):
        condition &= df_combined['고객번호'].str.contains(request.form.get('customer_number'), case=False)
    if request.form.get('master_number'):
        condition &= df_combined['청구번호'].str.contains(request.form.get('master_number'), case=False)
    if request.form.get('contract_number'):
        condition &= df_combined['계약번호'].str.contains(request.form.get('contract_number'), case=False)

    filtered_data = df_combined.loc[condition, selected_columns]
    output_file = 'filtered_data.xlsx'
    filtered_data.to_excel(output_file, index=False)

    return send_file(output_file, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)


Master Columns: Index(['순번', '미납관리본부', '미납관리부서', '현행관리본부', '현행관리지사', '청구마스터번호', '고객번호', '계약번호',
       '청구멤버고객명', '청구멤버상호명', '서비스개시일', '신규/유지', '고객상태', '수납방법', '선후납', '설치주소',
       '영업관리번호', '영업자', '체납개월', 'BM', ' 체납액 ', ' 회수액 ', ' 잔존체납액 '],
      dtype='object')
Sub1 Columns: Index(['실입금일자', '청구번호', '상호', '입금액', '은행명', '계좌번호', '입금자명', '청구본부', '청구지사'], dtype='object')
0     2024.01.01  41214046   탁종합관리주식회사  165000         국민은행  60959014637214   
1     2024.01.01  41232935  (주)동인아이티에스   83600         국민은행  60959014718719   
2     2024.01.01  41403840      베로니스명가   44000         국민은행  61039014519158   
3     2024.01.01  41638416  세븐일레븐일산태영점   58015         국민은행  48359073374152   
4     2024.01.01  41803720      이상훈농장2  119880  농협은행(농협중앙회)  79018903262071   
...          ...       ...         ...     ...          ...             ...   
8209  2024.05.26  41566740    노틀담 몬테소리   83600         하나은행  35592382076937   
8210  2024.05.26  41603925          더빌   66000         국민은행  48359073247902

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 7, in <module>
    from .connect import *
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 12, in <module>
    import jupyter_client
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 8, in <module>
    from .asynchronous import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/jupyter_client/asynchronous/__init__.py", line 1, in <module>
    from .client import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "

SystemExit: 1

/Users/heebonpark/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
from flask import Flask, render_template, request, send_file
import pandas as pd
import os

app = Flask(__name__)

# 파일 경로 설정
file_paths = {
    "master": "/Users/heebonpark/Downloads/붙임4_체납활동전체DB.cs",
    "sub1": "/Users/heebonpark/Downloads/가상계좌병합.cs",
    # "sub2": "/Users/heebonpark/Downloads/5월_CMS출금_병합.cs",
    # "sub3": "/Users/heebonpark/Downloads/5월채무불이행정보예정등록.cs",
    # "sub4": "/Users/heebonpark/Downloads/5월3차 전사 과오납 보증금환급.cs"
}

# CSV 파일 읽기, 모든 데이터를 문자열로 처리
df_master = pd.read_csv(file_paths["master"], encoding='cp949', dtype=str)
df_sub1 = pd.read_csv(file_paths["sub1"], encoding='cp949', dtype=str, header=1)  # 2번째 행을 헤더로 사용
# df_sub2 = pd.read_csv(file_paths["sub2"], encoding='cp949', dtype=str)
# df_sub3 = pd.read_csv(file_paths["sub3"], encoding='cp949', dtype=str)
# df_sub4 = pd.read_csv(file_paths["sub4"], encoding='cp949', dtype=str)

# NaN 값을 빈 문자열로 대체
df_master.fillna('', inplace=True)
df_sub1.fillna('', inplace=True)
# df_sub2.fillna('', inplace=True)
# df_sub3.fillna('', inplace=True)
# df_sub4.fillna('', inplace=True)

# 데이터프레임 열 이름 확인
print("Master Columns:", df_master.columns)
print("Sub1 Columns:", df_sub1.columns)

# 필수 열 정의 (열 이름을 실제 데이터프레임의 열 이름으로 수정)
required_columns = ['고객번호', '청구번호', '계약번호']

# 데이터프레임 병합 함수 정의
def merge_dataframes(master_df, *sub_dfs):
    merged_df = master_df.copy()
    for sub_df in sub_dfs:
        existing_columns = [col for col in required_columns if col in master_df.columns and col in sub_df.columns]
        if existing_columns:
            merged_df = pd.merge(merged_df, sub_df, on=existing_columns, how='left')
        else:
            print(f"Warning: No matching columns found for merging with {sub_df}")
    return merged_df

# 데이터프레임 병합
df_combined = merge_dataframes(df_master, df_sub1)
# df_combined = merge_dataframes(df_master, df_sub1, df_sub2, df_sub3, df_sub4)

@app.route('/')
def index():
    columns = df_combined.columns.tolist()
    return render_template('index.html', columns=columns)

@app.route('/filter', methods=['POST'])
def filter_data():
    customer_number = request.form.get('customer_number')
    master_number = request.form.get('master_number')
    contract_number = request.form.get('contract_number')
    selected_columns = request.form.getlist('columns')

    condition = True
    if customer_number:
        condition &= df_combined['고객번호'].str.contains(customer_number, case=False)
    if master_number:
        condition &= df_combined['청구번호'].str.contains(master_number, case=False)
    if contract_number:
        condition &= df_combined['계약번호'].str.contains(contract_number, case=False)

    filtered_data = df_combined.loc[condition, selected_columns]

    return filtered_data.to_html(classes='styledTable', index=False)

@app.route('/download', methods=['POST'])
def download():
    selected_columns = request.form.getlist('columns')
    condition = True
    if request.form.get('customer_number'):
        condition &= df_combined['고객번호'].str.contains(request.form.get('customer_number'), case=False)
    if request.form.get('master_number'):
        condition &= df_combined['청구번호'].str.contains(request.form.get('master_number'), case=False)
    if request.form.get('contract_number'):
        condition &= df_combined['계약번호'].str.contains(request.form.get('contract_number'), case=False)

    filtered_data = df_combined.loc[condition, selected_columns]
    output_file = 'filtered_data.xlsx'
    filtered_data.to_excel(output_file, index=False)

    return send_file(output_file, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)


Master Columns: Index(['순번', '미납관리본부', '미납관리부서', '현행관리본부', '현행관리지사', '청구마스터번호', '고객번호', '계약번호',
       '청구멤버고객명', '청구멤버상호명', '서비스개시일', '신규/유지', '고객상태', '수납방법', '선후납', '설치주소',
       '영업관리번호', '영업자', '체납개월', 'BM', ' 체납액 ', ' 회수액 ', ' 잔존체납액 '],
      dtype='object')
Sub1 Columns: Index(['실입금일자', '청구번호', '상호', '입금액', '은행명', '계좌번호', '입금자명', '청구본부', '청구지사'], dtype='object')
0     2024.01.01  41214046   탁종합관리주식회사  165000         국민은행  60959014637214   
1     2024.01.01  41232935  (주)동인아이티에스   83600         국민은행  60959014718719   
2     2024.01.01  41403840      베로니스명가   44000         국민은행  61039014519158   
3     2024.01.01  41638416  세븐일레븐일산태영점   58015         국민은행  48359073374152   
4     2024.01.01  41803720      이상훈농장2  119880  농협은행(농협중앙회)  79018903262071   
...          ...       ...         ...     ...          ...             ...   
8209  2024.05.26  41566740    노틀담 몬테소리   83600         하나은행  35592382076937   
8210  2024.05.26  41603925          더빌   66000         국민은행  48359073247902

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 7, in <module>
    from .connect import *
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 12, in <module>
    import jupyter_client
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 8, in <module>
    from .asynchronous import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/heebonpark/anaconda3/lib/python3.11/site-packages/jupyter_client/asynchronous/__init__.py", line 1, in <module>
    from .client import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "

SystemExit: 1

/Users/heebonpark/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
mkdir templates
